# Service

In this exercise, you will create a Pod and expose it outside the cluster using a NodePort Service.

1. Create a namespace *sandbox*.

2. Configure kubectl to stay on this namespace.

3. Create a pod with these properties:
   
   - image: jwilder/whoami
   
   - name: jwilder/whoami
   
   - port: 8000
   
   - replicas: 3

4. Create a service:
   
   - type: nodePort
   
   - nodePort: 31001

5. Browse to your application with your browser.

6. Delete your service. Expose your deployment and create an ingress in order to expose your pods outside the cluster.
   
   - hint* : add the following line to the bottom of the `/etc/hosts` file.
   ```shell
    <ingress-ip-adress> <dns-name>
    # nginx-ingress 
    ```

7. Browse to your application with your browse to check the config.

8. Clean all created objects.

## Correction

1. ```shell
   kubectl create namespace sandbox;
   ```

2. ```bash
   kubectl config set-context --current --namespace=sandbox
   ```

3. ```yaml
   apiVersion: apps/v1
   kind: Deployment
   metadata:
     name: whoami
     namespace: sandbox
   spec:
     replicas: 3
     selector:
       matchLabels:
         app: whoami
     template:
       metadata:
         labels:
           app: whoami
       spec:
         containers:
         - name: whoami
           image: jwilder/whoami
           ports:
           - containerPort: 8000
   ```

4. ```yamlag-0-1el3e4rojag-1-1el3e4roj
   apiVersion: v1
   kind: Service
   metadata:
     name: whoami-service
     namespace: sandbox
     labels:
       app: whoami
   spec:
     type: NodePort
     selector:
       app: whoami
     ports:
       - port: 8000
         targetPort: 8000
         nodePort: 31000
   ```

5. ```shell
   kubectl get nodes -o wide
   NAME    STATUS   ROLES    AGE    VERSION   INTERNAL-IP     EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
   node1   Ready    master   2d4h   v1.19.3   10.162.201.13   <none>        Ubuntu 20.04.1 LTS   5.4.0-51-generic   docker://19.3.13
   node2   Ready    <none>   2d4h   v1.19.3   10.162.201.36   <none>        Ubuntu 20.04.1 LTS   5.4.0-51-generic   docker://19.3.13
   node3   Ready    <none>   2d4h   v1.19.3   10.162.201.77   <none>        Ubuntu 20.04.1 LTS   5.4.0-51-generic   docker://19.3.13
   ```
   
   *In my case*: 10.162.201.36:31000 or 10.162.201.77:31000. Not 10.162.201.13, this the master and no pod can be scheduled on it (for the moment).

6. ```shell
   kubectl delete -f service.yml 
   service "whoami-service" deleted
   ```
   
   ```shell
   kubectl expose deployment whoami --name whoami --type NodePort --protocol TCP --port 8000 --target-port 8000 --selector='app=whoami'
   ```
   
   ```shell
   kubectl describe service whoami
   Name:                     whoami
   Namespace:                sandbox
   Labels:                   <none>
   Annotations:              <none>
   Selector:                 app=whoami
   Type:                     NodePort
   IP:                       10.104.172.71
   Port:                     <unset>  8000/TCP
   TargetPort:               8000/TCP
   NodePort:                 <unset>  30325/TCP
   Endpoints:                10.32.0.5:8000,10.40.0.2:8000,10.40.0.3:8000
   Session Affinity:         None
   External Traffic Policy:  Cluster
   ```
   
   Browse to 10.162.201.36:30325

7. ```shell
   kubectl delete namespace sandbox;
   ```